# Convert Calcium Imaging data from .mat to NWB file
More details on [NWB Calcium imaging data](https://pynwb.readthedocs.io/en/stable/tutorials/domain/ophys.html#calcium-imaging-data).

**0.** We start importing the relevant modules to read from .mat file and to manipulate NWB file groups and datasets

In [ ]:
from datetime import datetime
from dateutil.tz import tzlocal
from pynwb import NWBFile, NWBHDF5IO, ProcessingModule
from pynwb.ophys import TwoPhotonSeries, OpticalChannel, ImageSegmentation, Fluorescence, DfOverF, MotionCorrection
from pynwb.device import Device
from pynwb.base import TimeSeries

import scipy.io
import numpy as np
import h5py
import os

**1.** Load the .mat files containing calcium imaging data

In [ ]:
path_to_files = '/Users/bendichter/Desktop/Axel Lab/data' #r'C:\Users\Luiz\Desktop\Axel'

# Open info file
fname0 = 'fly2_run1_info.mat'
fpath0 = os.path.join(path_to_files, fname0)
f_info = scipy.io.loadmat(fpath0, struct_as_record=False, squeeze_me=True)
info = f_info['info']

# Open .mat file containing Calcium Imaging data
fname1 = '2019_04_18_Nsyb_NLS6s_Su_walk_G_fly2_run1_8401reg.mat'
fpath1 = os.path.join(path_to_files, fname1)
file = h5py.File(fpath1, 'r')
options = file['options']
landmarkThreshold = file['landmarkThreshold']
templates = file['templates']
R = file['R']
Y = file['Y']

**2.** Create a new [NWB file instance](https://pynwb.readthedocs.io/en/stable/pynwb.file.html#pynwb.file.NWBFile), fill it with all the relevant information

In [ ]:
#Create new NWB file
nwb = NWBFile(session_description='my CaIm recording', 
              identifier='EXAMPLE_ID', 
              session_start_time=datetime.now(tzlocal()),
              experimenter='Dr. ABC',
              lab='My Lab',
              institution='My University',
              experiment_description='Some description.',
              session_id='IDX')
print(nwb)

**3.** Create [Device](https://pynwb.readthedocs.io/en/stable/pynwb.device.html#pynwb.device.Device) and [OpticalChannel](https://pynwb.readthedocs.io/en/stable/pynwb.ophys.html#pynwb.ophys.OpticalChannel) containers to be used by a specific [ImagingPlane](https://pynwb.readthedocs.io/en/stable/pynwb.ophys.html#pynwb.ophys.ImagingPlane).

In [ ]:
#Create and add device
device = Device(info.objective.replace('/','_'))
nwb.add_device(device)

# Create an Imaging Plane for Yellow
optical_channel_Y = OpticalChannel(name='optical_channel_Y',
                                   description='2P Optical Channel',
                                   emission_lambda=510.)
imaging_plane_Y = nwb.create_imaging_plane(name='imaging_plane_Y',
                                           optical_channel=optical_channel_Y,
                                           description='Imaging plane',
                                           device=device,
                                           excitation_lambda=488., 
                                           imaging_rate=info.daq.scanRate,
                                           indicator='NLS-GCaMP6s',
                                           location='whole central brain')

# Create an Imaging Plane for Red
optical_channel_R = OpticalChannel(name='optical_channel_R',
                                   description='2P Optical Channel',
                                   emission_lambda=633.)
imaging_plane_R = nwb.create_imaging_plane(name='imaging_plane_R',
                                           optical_channel=optical_channel_R,
                                           description='Imaging plane',
                                           device=device,
                                           excitation_lambda=488., 
                                           imaging_rate=info.daq.scanRate,
                                           indicator='redStinger',
                                           location='whole central brain')

print(nwb)

**4.** Create a [TwoPhotonSeries](https://pynwb.readthedocs.io/en/stable/pynwb.ophys.html#pynwb.ophys.TwoPhotonSeries) container to store the raw data. Raw data usually goes on the `acquisition` group of NWB files.

In [ ]:
#Stores raw data in acquisition group - dims=(X,Y,Z,T)
raw_image_series_Y = TwoPhotonSeries(name='TwoPhotonSeries_Y', 
                                     imaging_plane=imaging_plane_Y,
                                     rate=info.daq.scanRate,
                                     dimension=[36, 167, 257],
                                     data=Y[:,:,:,:]) 

raw_image_series_R = TwoPhotonSeries(name='TwoPhotonSeries_R', 
                                     imaging_plane=imaging_plane_R,
                                     rate=info.daq.scanRate,
                                     dimension=[36, 167, 257],
                                     data=R[:,:,:,:]) 

nwb.add_acquisition(raw_image_series_Y)
nwb.add_acquisition(raw_image_series_R)

print(nwb.acquisition['TwoPhotonSeries'])

**5.** A very important data preprocessing step for calcium signals is motion correction. We can store the processed result data in the [MotionCorrection](https://pynwb.readthedocs.io/en/stable/pynwb.ophys.html#pynwb.ophys.MotionCorrection) container, inside the `processing` group of NWB files.

In [ ]:
#Creates ophys ProcessingModule and add to file
ophys_module = ProcessingModule(name='ophys',
                                description='contains optical physiology processed data.')
nwb.add_processing_module(ophys_module)

#Stores corrected data in TwoPhotonSeries container
corrected_image_series = TwoPhotonSeries(name='TwoPhotonSeries_corrected', 
                                         imaging_plane=imaging_plane,
                                         rate=info.daq.scanRate,
                                         dimension=[36, 167, 257],
                                         data=Y[:,:,:,0])

#TimeSeries XY translation correction values
xy_translation = TimeSeries(name='xy_translation', 
                            data=np.zeros((257,2)),
                            rate=info.daq.scanRate)

#Adds the corrected image stack to MotionCorrection container
motion_correction = MotionCorrection()
motion_correction.create_corrected_image_stack(corrected=corrected_image_series, 
                                               original=raw_image_series, 
                                               xy_translation=xy_translation)

#Add MotionCorrection to processing group
ophys_module.add_data_interface(motion_correction)

**6.** Any processed data should be stored in the `processing` group of NWB files. A list of available containers can be found [here](https://pynwb.readthedocs.io/en/stable/overview_nwbfile.html#processing-modules). These include, for example, [DfOverF](https://pynwb.readthedocs.io/en/stable/pynwb.ophys.html#pynwb.ophys.DfOverF), [ImageSegmentation](https://pynwb.readthedocs.io/en/stable/pynwb.ophys.html#pynwb.ophys.ImageSegmentation), [Fluorescence](https://pynwb.readthedocs.io/en/stable/pynwb.ophys.html#pynwb.ophys.Fluorescence) and others.

In [ ]:
#Stores processed data of different types in ProcessingModule group
#Image segmentation
img_seg = ImageSegmentation()
ophys_module.add_data_interface(img_seg)

#Fluorescence
fl = Fluorescence()
ophys_module.add_data_interface(fl)

#DfOverF
dfoverf = DfOverF()
ophys_module.add_data_interface(dfoverf)

print(nwb.processing['ophys'])

**7.** The NWB structure is is place, but we still need to save it to file:

In [ ]:
#Saves to NWB file
fname_nwb = 'file_1.nwb'
fpath_nwb = os.path.join(path_to_files, fname_nwb)
with NWBHDF5IO(fpath_nwb, mode='w') as io:
    io.write(nwb)
print('File saved with size: ', os.stat(fpath_nwb).st_size/1e6, ' mb')

**8.** Finally, let's load it and check the file contents:

In [ ]:
#Loads NWB file
with NWBHDF5IO(fpath_nwb, mode='r') as io:
    nwb = io.read()
    print(nwb)
    print(nwb.processing['ophys'].data_interfaces['MotionCorrection'])

# Evan Schaffer data
from `.npz` files to NWB.

In [ ]:
from datetime import datetime
from dateutil.tz import tzlocal
from pynwb import NWBFile, NWBHDF5IO, ProcessingModule
from pynwb.ophys import TwoPhotonSeries, OpticalChannel, ImageSegmentation, Fluorescence, DfOverF, MotionCorrection
from pynwb.device import Device
from pynwb.base import TimeSeries

import scipy.io
import numpy as np
import h5py
import os
import matplotlib.pyplot as plt

a = np.load('2019_07_01_Nsyb_NLS6s_walk_fly2.npz')
print('First file:')
print('Groups:', a.files)
print('Dims (height,width,depth):', a['dims'])
print('dFF shape: ', a['dFF'].shape)

b = np.load('2019_07_01_Nsyb_NLS6s_walk_fly2_A.npz')
print('     ')
print('Second file - Sparse Matrix:')
print('Groups:', b.files)
print('Indices: ', b['indices'], '  | Shape: ',b['indices'].shape)
print('Indptr: ', b['indptr'], '  | Shape: ',b['indptr'].shape)
print('Format: ', b['format'])
print('Shape: ', b['shape'])
print('Data: ', b['data'], '  | Shape: ',b['data'].shape)

Start creating a new NWB file instance and populating it with fake raw data

In [ ]:
#Create new NWB file
nwb = NWBFile(session_description='my CaIm recording', 
              identifier='EXAMPLE_ID', 
              session_start_time=datetime.now(tzlocal()),
              experimenter='Dr. ABC',
              lab='My Lab',
              institution='My University',
              experiment_description='Some description.',
              session_id='IDX')

#Create and add device
device = Device('MyDevice')
nwb.add_device(device)

# Create an Imaging Plane for Yellow
optical_channel = OpticalChannel(name='optical_channel',
                                 description='2P Optical Channel',
                                 emission_lambda=510.)
imaging_plane = nwb.create_imaging_plane(name='imaging_plane',
                                         optical_channel=optical_channel,
                                         description='Imaging plane',
                                         device=device,
                                         excitation_lambda=488., 
                                         imaging_rate=1000.,
                                         indicator='NLS-GCaMP6s',
                                         location='whole central brain',
                                         conversion=1.0)

#Stores raw data in acquisition group - dims=(X,Y,Z,T)
Xp = a['dims'][0][0]
Yp = a['dims'][0][1]
Zp = a['dims'][0][2]
T = a['dFF'].shape[1]
nCells = a['dFF'].shape[0]
fake_data = np.random.randn(Xp,Yp,Zp,100)
raw_image_series = TwoPhotonSeries(name='TwoPhotonSeries', 
                                   imaging_plane=imaging_plane,
                                   rate=1000.,
                                   dimension=[Xp,Yp,Zp],
                                   data=fake_data) 
nwb.add_acquisition(raw_image_series)

#Creates ophys ProcessingModule and add to file
ophys_module = ProcessingModule(name='ophys',
                                description='contains optical physiology processed data.')
nwb.add_processing_module(ophys_module)

Now transform the lists of indices into (xp,yp,zp) masks. With the masks created, we can add them to a plane segmentation class.

In [ ]:
def make_voxel_mask(indices, dims):
    """
    indices - List with voxels indices, e.g. [64371, 89300, 89301, ..., 3763753, 3763842, 3763843]
    dims - (height, width, depth) in pixels
    """
    voxel_mask = []
    for ind in indices:
        zp = np.floor(ind/(dims[0]*dims[1])).astype('int')
        rest = ind%(dims[0]*dims[1])
        yp = np.floor(rest/dims[0]).astype('int')
        xp = rest%dims[0]
        voxel_mask.append((xp,yp,zp,1))
    
    return voxel_mask 
        
#Call function
indices = b['indices']
indptr = indices[b['indptr'][0:-1]]
dims = np.squeeze(a['dims'])
voxel_mask = make_voxel_mask(indptr, dims)

#Create Image Segmentation compartment
img_seg = ImageSegmentation()
ophys_module.add_data_interface(img_seg)

#Create plane segmentation and add ROIs
ps = img_seg.create_plane_segmentation(description='plane segmentation',
                                       imaging_plane=imaging_plane, 
                                       reference_images=raw_image_series)
ps.add_roi(voxel_mask=voxel_mask)

With the ROIs created, we can add the dF/F data

In [ ]:
#DFF measures
dff = DfOverF(name='dff_interface')
ophys_module.add_data_interface(dff)

#create ROI regions
roi_region = ps.create_roi_table_region(description='ROI table region', 
                                        region=[0])

#create ROI response series
dff_data = a['dFF']
dFF_series = dff.create_roi_response_series(name='df_over_f',
                                            data=dff_data,
                                            unit='NA',
                                            rois=roi_region,
                                            rate=1000.)

The file contains two arrays with pixel indexing: one with all pixels at cells and one with only one reference pixel per cell. Let's see how it looks like in a 3D plot:

In [ ]:
#3D scatter with masks points
from mpl_toolkits.mplot3d import Axes3D

#Reference points
xptr = [p[0] for p in voxel_mask]
yptr = [p[1] for p in voxel_mask]
zptr = [p[2] for p in voxel_mask]

#All points in mask
all_pt_mask = make_voxel_mask(indices, dims)
x = [p[0] for p in all_pt_mask]
y = [p[1] for p in all_pt_mask]
z = [p[2] for p in all_pt_mask]

%matplotlib notebook
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(x, y, z, c='k', marker='.', s=.5)
ax.scatter(xptr, yptr, zptr, c='r', marker='o', s=20)

In [ ]:
#Saves to NWB file
path_to_files = ''
fname_nwb = 'file_1.nwb'
fpath_nwb = os.path.join(path_to_files, fname_nwb)
with NWBHDF5IO(fpath_nwb, mode='w') as io:
    io.write(nwb)
print('File saved with size: ', os.stat(fpath_nwb).st_size/1e6, ' mb')

In [ ]:
#Loads NWB file
with NWBHDF5IO(fpath_nwb, mode='r') as io:
    nwb = io.read()
    print(nwb)
    print(nwb.processing['ophys'].data_interfaces['dff_interface'].roi_response_series['df_over_f'])